In [15]:
import pandas as pd
import matplotlib.pyplot as plt

#load the forecast files
df_prb = pd.read_csv("/content/drive/MyDrive/forecast_DL_Prb_Utilization_Prophet.csv")
df_ue = pd.read_csv("/content/drive/MyDrive/forecast_Avg_UE_Number_Prophet.csv")

#convert timestamp to datetime
df_prb["Timestamp"] = pd.to_datetime(df_prb["Timestamp"])
df_ue["Timestamp"] = pd.to_datetime(df_ue["Timestamp"])

#merge both KPI forecasts on Timestamp and nCI
merged = pd.merge(df_prb, df_ue, on=["Timestamp", "nCI"], suffixes=("_prb", "_ue"))

#ask user for ES Mode
mode = input("Enter ES mode (Conservative / Moderate / Aggressive): ").capitalize()

#define thresholds for ES modes
thresholds = {
    "Conservative": {"DL_Prb_Utilization": 10, "Avg_UE_Number": 10},
    "Moderate": {"DL_Prb_Utilization": 15, "Avg_UE_Number": 13},
    "Aggressive": {"DL_Prb_Utilization": 20, "Avg_UE_Number": 17}
}

if mode not in thresholds:
    print("Invalid mode. Using Moderate.")
    mode = "Moderate"

#use thresholds
prb_thresh = thresholds[mode]["DL_Prb_Utilization"]
ue_thresh = thresholds[mode]["Avg_UE_Number"]

#filter data below threshold
filtered = merged[
    (merged["Forecast_prb"] <= prb_thresh) &
    (merged["Forecast_ue"] <= ue_thresh)
]
print("DL_Prb_Utilization Forecast Summary:")
print(merged["Forecast_prb"].describe())

print("\nAvg_UE_Number Forecast Summary:")
print(merged["Forecast_ue"].describe())
print(f"Filtered rows count: {len(filtered)}")

#final recommendations per overlapping window
window_recommendations = []

for cell in filtered["nCI"].unique():
    cell_data = filtered[filtered["nCI"] == cell].sort_values("Timestamp").copy()
    cell_data["Gap"] = cell_data["Timestamp"].diff().dt.total_seconds().div(60)
    cell_data["Group"] = (cell_data["Gap"].fillna(15).round() != 15).cumsum()


    for _, group in cell_data.groupby("Group"):
        if len(group) >= 2:  #minimum 2 intervals=30 mins
            start_time = group["Timestamp"].min()
            end_time = group["Timestamp"].max()
            duration = (end_time - start_time).seconds // 60 + 15  #inclusive of last interval
            if duration >= 30:
                window_recommendations.append({
                    "nCI": cell,
                    "Start_Time": start_time,
                    "End_Time": end_time,
                    "Duration_Minutes": duration,
                    "Recommendation": "Shut Down (Low Utilization)"
                })

#create DataFrame
rec_df = pd.DataFrame(window_recommendations)

#save to file with mode in filename
rec_df.to_csv(f"energy_saving_recommendations_{mode.lower()}.csv", index=False)
print(f"Saved detailed recommendations to energy_saving_recommendations_{mode.lower()}.csv")

Enter ES mode (Conservative / Moderate / Aggressive): aggressive
DL_Prb_Utilization Forecast Summary:
count    105984.000000
mean         46.376731
std          20.536641
min          13.891410
25%          27.132605
50%          47.073500
75%          57.124689
max          86.802106
Name: Forecast_prb, dtype: float64

Avg_UE_Number Forecast Summary:
count    105984.000000
mean         22.091657
std          12.049200
min           1.941334
25%          11.592925
50%          23.280939
75%          29.810255
max          42.943586
Name: Forecast_ue, dtype: float64
Filtered rows count: 15141
Saved detailed recommendations to energy_saving_recommendations_aggressive.csv
